# Hierarchical Dirichlet Process Gamma Mixture Model for Terahertz Band Wireless Communication Channel Modelling 

In [ ]:
import arviz as az
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy as sp
import seaborn as sns

import pymc3.distributions.transforms as tr

from matplotlib import pyplot as plt
from theano import tensor as tt
from scipy.io import savemat

print(f"Running on PyMC3 v{pm.__version__}")

In [ ]:
%config InlineBackend.figure_format = 'retina'
SEED = 8927
rng = np.random.default_rng(SEED)
az.style.use("arviz-darkgrid")

In [ ]:
THz_received_power = pd.read_csv("THz_measurement_data.csv")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

n_bins = 50
ax.hist(THz_received_power.twenty_cm, bins=n_bins, color="blue", lw=0, alpha=0.5)

ax.set_xlabel("P_rx")
ax.set_ylabel("Occurance");

In [ ]:
# Truncation number
K=30

In [ ]:
def stick_breaking(beta):
    portion_remaining = tt.concatenate([[1], tt.extra_ops.cumprod(1 - beta)[:-1]])
    return beta * portion_remaining

In [ ]:
# Hyper-priors
var_phi = 1
var_rho = 1
epsilon_= 1
mu_     = 1

var_theta = 1
var_pi    = 1
zeta_     = 0.001

In [ ]:
# model 
with pm.Model() as model: 
    alpha = pm.Gamma("alpha", 1.0, 1.0)
    beta = pm.Beta("beta", 1.0, alpha, shape=K)
    w = pm.Deterministic("w", stick_breaking(beta))

    upsilon_ = pm.Gamma("upsilon_", alpha = var_phi , beta = var_rho , shape = K )
    nu_      = pm.InverseGamma("nu_", alpha = epsilon_ , beta = mu_ , shape = K )
    beta_    = pm.Gamma("beta_", alpha = nu_ , beta = upsilon_ , shape = K )

    lambda_  = pm.InverseGamma("lambda_", alpha = var_theta, beta= var_pi, shape = K )
    kappa_   = pm.Exponential("kappa_",    lam   = zeta_, shape = K)
    alpha_   = pm.InverseGamma("alpha_", alpha = lambda_, beta = kappa_, shape = K )
    
    components = pm.Gamma.dist(alpha=alpha_, beta=beta_, shape = K )
    x_obs = pm.Mixture("x_obs", w, comp_dists=components, observed=THz_received_power.twenty_cm.values)

In [ ]:
# Show model structure
pm.model_to_graphviz(model)

In [ ]:
# Different options can be applied.
with model:
    trace = pm.sample(.................................)

In [ ]:
x_plot = np.linspace(4, 11.5, 4096)
n_components_used = np.apply_along_axis(lambda x: np.unique(x).size, 1, trace['component'])
fig, ax = plt.subplots(figsize=(8, 6))

bins = np.arange(n_components_used.min(), n_components_used.max() + 1)
ax.hist(n_components_used + 1, bins=bins, density=True, lw=0, alpha=0.75);

ax.set_xticks(bins + 0.5);
ax.set_xticklabels(bins);
ax.set_xlim(bins.min(), bins.max() + 1);
ax.set_xlabel('Number of mixture components used');

ax.set_ylabel('Posterior probability');

post_pdf_contribs = sp.stats.gamma.pdf(
    x=np.atleast_3d(x_plot),
    a=trace["alpha_"][:, np.newaxis, :],
    scale=1/trace["beta_"][:, np.newaxis, :],
)
post_pdfs = (trace['w'][:, np.newaxis, :] * post_pdf_contribs).sum(axis=-1)

post_pdf_low, post_pdf_high = np.percentile(post_pdfs, [2.5, 97.5], axis=0)


fig, ax = plt.subplots(figsize=(8, 6))

n_bins = 20
ax.hist(THz_received_power.twenty_cm.values, bins=n_bins, density=True,
        color="blue", lw=0, alpha=0.5);

ax.fill_between(x_plot, post_pdf_low, post_pdf_high,
                color='gray', alpha=0.45);
ax.plot(x_plot, post_pdfs[0],
        c='gray', label='Posterior sample densities');
ax.plot(x_plot, post_pdfs[::100].T, c='gray');
ax.plot(x_plot, post_pdfs.mean(axis=0),
        c='k', label='Posterior expected density');

ax.set_xlabel('Standardized waiting time between eruptions');

ax.set_yticklabels([]);
ax.set_ylabel('Density');

ax.legend(loc=2);


fig, ax = plt.subplots(figsize=(8, 6))

n_bins = 20
ax.hist(THz_received_power.twenty_cm.values, bins=n_bins, density=True,
        color="blue", lw=0, alpha=0.5);

ax.plot(x_plot, post_pdfs.mean(axis=0),
        c='k', label='Posterior expected density');
ax.plot(x_plot, (trace['w'][:, np.newaxis, :] * post_pdf_contribs).mean(axis=0)[:, 0],
        '--', c='k', label='Posterior expected mixture\ncomponents\n(weighted)');
ax.plot(x_plot, (trace['w'][:, np.newaxis, :] * post_pdf_contribs).mean(axis=0),
        '--', c='k');

ax.set_xlabel('Standardized waiting time between eruptions');

ax.set_yticklabels([]);
ax.set_ylabel('Density');

ax.legend(loc=2);

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

plot_w = np.arange(K) + 1

ax.bar(plot_w-0.5 , trace["w"].mean(axis=0), width=1.0, lw=0)

ax.set_xlim(0.5, K)
ax.set_xlabel("Component")

ax.set_ylabel("Posterior expected mixture weight");

print("weights shape =", trace["w"].shape)

In [ ]:
x_plot = np.linspace(4, 11.5, 4096)
post_pdf_contribs = sp.stats.gamma.pdf(
    x=np.atleast_3d(x_plot),
    a=trace["alpha_"][:, np.newaxis, :],
    scale=1/trace["beta_"][:, np.newaxis, :],
)
post_pdfs = (trace["w"][:, np.newaxis, :] * post_pdf_contribs).sum(axis=-1)

post_pdf_low, post_pdf_high = np.percentile(post_pdfs, [2.5, 97.5], axis=0)
post_pdf_contribs.shape

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

n_bins = 50
ax.hist(THz_received_power.twenty_cm.values, bins=n_bins, density=True, color="blue", lw=0, alpha=0.5)

ax.fill_between(x_plot, post_pdf_low, post_pdf_high, color="gray", alpha=0.45)
ax.plot(x_plot, post_pdfs[0],"-", c="gray", label="Posterior sample densities")
ax.plot(x_plot, post_pdfs[::10].T,"-", c="gray")
ax.plot(x_plot, post_pdfs.mean(axis=0), "--", c="blue", label="Posterior expected density")

ax.set_xlabel("P_rx")

ax.set_yticklabels([])
ax.set_ylabel("Density")

ax.legend(loc=2);

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

n_bins = 20
ax.hist(THz_received_power.twenty_cm.values, bins=n_bins, density=True, color="blue", lw=0, alpha=0.5)

ax.plot(x_plot, post_pdfs.mean(axis=0), c="k", label="Posterior expected density")
ax.plot(
    x_plot,
    (trace["w"][:, np.newaxis, :] * post_pdf_contribs).mean(axis=0)[:, 0],
    "--",
    c="k",
    label="Posterior expected mixture\ncomponents\n(weighted)",
)
ax.plot(x_plot, (trace["w"][:, np.newaxis, :] * post_pdf_contribs).mean(axis=0), "--", c="k")

ax.set_xlabel("P_rx 20 cm")

ax.set_yticklabels([])
ax.set_ylabel("Density")

ax.legend(loc=2);

In [ ]:
mat_file = {  
                "generated_mixture_data" : THz_received_power.twenty_cm.values, 
                "x_plot" : x_plot, 
                "w_trace" : trace["w"][:, np.newaxis, :], 
                "alpha_trace" : trace["alpha_"][:, np.newaxis, :], 
                "beta_trace" : trace["beta_"][:, np.newaxis, :], 
                "post_pdf_contribs": post_pdf_contribs, 
                "weighted_post_pdf_contribs": (trace["w"][:, np.newaxis, :] * post_pdf_contribs), 
                "weighted_post_pdf_contribs_means": (trace["w"][:, np.newaxis, :] * post_pdf_contribs).mean(axis=0),  
                "post_pdfs":  post_pdfs, 
                "post_pdfs_means":   post_pdfs.mean(axis=0)  }

savemat("mat_file.mat", mat_file)